# Hello World! Inference - Bring Your Own Pickle File 'Model'

In [ ]:
!pip install dill

In [ ]:
import dill as pickle

### Create our Custom Hello World 'Model'
Here we create a python class called Model and inside it we have a method called predict that takes in a dataframe and returns "Hello World!" for each row in the dataframe. This can be thought of as training our model locally outside of SageMaker.

In [ ]:
class Model:
    def predict(x): 
        sentance = "Hello World! "* x.shape[0] 
        return sentance

### Pickle the Model. 
Now we want to serialize our custom 'Model'. 

In [ ]:
with open('hello-world-model.pkl', 'wb') as f:
    pickle.dump(Model, f)

### Load the model and test
Now we can deserialize the model and test it.

In [ ]:
with open('hello-world-model.pkl', 'rb') as f:
    model = pickle.load(f)

#### Predict locally

In [ ]:
import pandas as pd

raw_data = ['Say Hello World!', 'Say Hello World!', 'Say Hello World!', 'Say Hello World!']
df = pd.DataFrame(raw_data) 

model.predict(df)

### Package our Model to deploy to a SageMaker endpoint
SageMaker requires our Model to be tared and gzipped. 

In [ ]:
! tar -czvf hello_world_model.tar.gz hello-world-model.pkl

### Upload our Model to S3
Now we can upload our Model pickeled (serialized) Model to S3

In [ ]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
prefix = 'DEMO-hello-world'

    
! aws s3 cp hello_world_model.tar.gz s3://$bucket/$prefix/model.tar.gz

### Build and Push our container to ECR
We have our custom Model that is now in S3. All we need now is a container that implemenets the hosting requirements and inference logic.
An important file to look at is the predictor.py here we coded the logic to deserialize the Model and make a inference from it. SageMaker fetched our Model from S3 and placed it in /opt/ml/model/. Take a look at the get_model() method which uses the code above to load the model from the SageMaker model path. 

In [ ]:
!sed -n '26,31p' container/Files/predictor.py

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-hello-world-inference

cd container


chmod +x Files/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

### Deploy our Model to an Endpoint
Our container has been pushed to ECR and our Model is in S3 now we have everything we need to Deploy to a SageMaker Endpoint.

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import RealTimePredictor, csv_serializer, csv_deserializer

In [ ]:
# Create a Predictor so we can use the predict() method to invoke our 'model'.
class Predictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session=None):
        super(Predictor, self).__init__(
            endpoint_name, sagemaker_session, csv_serializer, csv_deserializer
        )

#### Create a SageMaker Model

In [ ]:
sess = sagemaker.Session()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-hello-world-inference:latest'.format(account, region)

sagemaker_model = Model(
                        sagemaker_session= sess,
                        model_data = "s3://"+bucket+"/"+prefix+"/model.tar.gz" , 
                        image_uri= image,
                        role=role,
                        predictor_cls= Predictor
                       )

#### Deploy the Model to an Endpoint

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count= 1,instance_type= 'ml.m4.xlarge' )

#### Get a prediction from our Endpoint

In [ ]:
predictor.predict("Say Hello World!")

#### Optional cleanup
When you're done with the endpoint, you'll want to clean it up.

In [ ]:
sess.delete_endpoint(predictor.endpoint)

## Batch Transform Job
Now that we have seen the we can deploy our custom pickle file to a RealTime Endpoint and get a prediction, lets now create a Batch Transform Job that will give us batch inference.

### Create the input data and upload it to S3

In [ ]:
%%writefile batchdata.csv
Say Hello World!
Say Hello World!
Say Hello World!
Say Hello World!
Say Hello World!
Say Hello World!
Say Hello World!
Say Hello World!
Say Hello World!

In [ ]:
! aws s3 cp batchdata.csv s3://$bucket/$prefix/batchdata.csv

### Create the Transfromer from the SageMaker Model and transform the data we created up.

In [ ]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}/{}".format(sess.default_bucket(),"DEMO-hello-world",transform_output_folder)

transformer = sagemaker_model.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

In [ ]:
input_path="s3://{}/{}/{}".format(sess.default_bucket(),"DEMO-hello-world","batchdata.csv")


transformer.transform(input_path, content_type='text/csv', split_type='Line')
transformer.wait()

### View the Batch Transform results.

In [ ]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "DEMO-hello-world/{}/batchdata.csv.out".format(transform_output_folder), '/tmp/batchdata.csv.out')


with open('/tmp/batchdata.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))